cmmcmcv

In [20]:
import pandas as pd
import numpy as np
import ast
# Load transition matrix
transition_df = pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\transition_probability_matrix.csv", index_col=0)

# Load emission matrix
emission_df = pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\emissive_probability_matrix.csv", index_col=0)

start_df= pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\start_probability_matrix.csv", index_col=0)

# Ensure index and column names are properly set
print("Transition Matrix Index:", transition_df.index)
print("Transition Matrix Columns:", transition_df.columns)


Transition Matrix Index: Index(['NUM', 'NOUN', 'DET', 'VERB', 'ADJ', '.', 'PUNCT', 'ADP', 'CONJ',
       'PROPN', 'X', 'PRON', 'ADV', 'AUX', 'INTJ', 'CCONJ', 'PRT', 'PART',
       'SCONJ', '_', 'SYM'],
      dtype='object')
Transition Matrix Columns: Index(['NUM', 'NOUN', 'DET', 'VERB', 'ADJ', '.', 'PUNCT', 'ADP', 'CONJ',
       'PROPN', 'X', 'PRON', 'ADV', 'AUX', 'INTJ', 'CCONJ', 'PRT', 'PART',
       'SCONJ', '_', 'SYM'],
      dtype='object')


In [27]:
def viterbi_algorithm(words, transition_df, emission_df, start_df):
    # words = sentence.split()
  
    pos_tags = list(transition_df.index)  # POS tag list
    num_tags = len(pos_tags)
    num_words = len(words)
    
    # Initialize DP tables
    viterbi = np.zeros((num_tags, num_words))  
    backpointer = np.zeros((num_tags, num_words), dtype=int)  
    
    # Initialization step
    for i, tag in enumerate(pos_tags):
        start_prob = start_df.loc[tag, 'START'] if tag in start_df.index else 1e-6
        emission_prob = emission_df.loc[tag, words[0]] if words[0] in emission_df.columns else 1e-6
        viterbi[i, 0] = start_prob * emission_prob
    
    # Recursion step
    for t in range(1, num_words):
        for i, tag in enumerate(pos_tags):
            max_prob, max_state = max(
                (viterbi[k, t-1] * transition_df.loc[prev_tag, tag] * (emission_df.loc[tag, words[t]] if words[t] in emission_df.columns else 1e-6), k)
                if prev_tag in transition_df.index else (1e-6, k)
                for k, prev_tag in enumerate(pos_tags)
            )
            viterbi[i, t] = max_prob
            backpointer[i, t] = max_state
    
    # Traceback
    best_path = []
    best_last_state = np.argmax(viterbi[:, -1])
    best_path.append(pos_tags[best_last_state])
    
    for t in range(num_words - 1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, pos_tags[best_last_state])
    
    return list(zip(words, best_path))

# # Example Usage
# sentence = "The industry has said"
# predicted_tags = viterbi_algorithm(sentence, transition_df, emission_df, start_df)
# print(predicted_tags)

In [24]:
test_data= pd.read_csv(r"C:\Users\mitta\OneDrive - iiit-b\Documents\NLP\Assignment1\dataset\TEST.csv", header=None, names=['sentence'])
test_data['sentence'] = test_data['sentence'].apply(ast.literal_eval)

In [28]:
def evaluate(test_data):
    correct, total = 0, 0
    for sentence in test_data['sentence']:
       
        words = [word for word,_ in sentence]
        
        actual_tags = [tag for _, tag in sentence]
        predicted_tags = viterbi_algorithm(words, transition_df, emission_df,start_df)
        correct += sum(p == a for p, a in zip(predicted_tags, actual_tags))
        total += len(actual_tags)
    accuracy = correct / total if total > 0 else 0
    print("Model Accuracy: "+str(accuracy*100)+"%")

evaluate(test_data)


KeyboardInterrupt: 